In [ ]:
#If opening in colab run this cell
!git clone https://github.com/CTVisMe/DS4B_Sp24.git
%cd DS4B_Sp24/ClassModules/Module6_Similarity/

Cloning into 'DS4B_Sp24'...
remote: Enumerating objects: 860, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 860 (delta 122), reused 255 (delta 116), pack-reused 598
Receiving objects: 100% (860/860), 159.30 MiB | 11.32 MiB/s, done.
Resolving deltas: 100% (385/385), done.
Updating files: 100% (146/146), done.
/content/DS4B_Sp24/ClassModules/Module6_Similarity


Universal Bank is a relatively young bank growing rapidly in terms of overall customer acquisition. The majority of these customers are liability customers (depositors) with varying sizes of relationship with the bank. The customer base of asset customers (borrowers) is quite small, and the bank is interested in expanding this base rapidly to bring in more loan business. In particular, it wants to explore ways of converting its liability customers to personal loan customers (while retaining them as depositors).

A campaign that the bank ran last year for liability customers showed a healthy conversion rate of over 9% success. This has encouraged the retail marketing department to devise smarter campaigns with better target marketing. The goal is to use k-NN to predict whether a new customer will accept a loan offer. This will serve as the basis for the design of a new campaign.

The file UniversalBank.csv contains data on 5000 customers. The data include customer demographic information (age, income, etc.), the customer’s relationship with the bank (mortgage, securities account, etc.), and the customer response to the last personal loan campaign (Personal Loan). Among these 5000 customers, only 480 (=9.6%) accepted the personal loan that was offered to them in the earlier campaign.

Partition the data into training (60%) and validation (40%) sets

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pylab as plt

ubank = pd.read_csv('data/UniversalBank.csv')


In [ ]:
ubank.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


Partition the data into train/test (80/20)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = ubank
y = ubank['Personal Loan']

X = pd.get_dummies(X, columns=['Education'], prefix='Education')

X = X.drop(['ID', 'ZIP Code', 'Personal Loan'], axis=1)

X.head()

,Age,Experience,Income,Family,CCAvg,Mortgage,Securities Account,CD Account,Online,CreditCard,Education_1,Education_2,Education_3
0,25,1,49,4,1.6,0,1,0,0,0,1,0,0
1,45,19,34,3,1.5,0,1,0,0,0,1,0,0
2,39,15,11,1,1.0,0,0,0,0,0,1,0,0
3,35,9,100,1,2.7,0,0,0,0,0,0,1,0
4,35,8,45,4,1.0,0,0,0,0,1,0,1,0


In [ ]:

# Dropping the ID and ZIP Code columns

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape


((4000, 13), (1000, 13), (4000,), (1000,))

Consider the following customer named Danah:

Age = 40, Experience = 10, Income = 84, Family = 2, CCAvg = 2, Education_1 = 0, Education_2 = 1, Education_3 = 0, Mortgage = 0, Securities Account = 0, CD Account = 0, Online = 1, and Credit Card = 1.

Perform a k-NN classification with all predictors except ID and ZIP code using k = 1. Transform categorical predictors with more than two categories into dummy variables first. Specify the success class as 1 (loan acceptance), and use the default cutoff value of 0.5.

How would Danah be classified?

What is a choice of k that balances between overfitting and ignoring the predictor information?

In [ ]:
danah_features = [40, 10, 84, 2, 2, 0, 0, 0, 1, 1, 0, 1, 0]

danah_features_scaled = scaler.transform([danah_features])

# Predicting Danah's classification with k=1

knn_k1 = KNeighborsClassifier(n_neighbors=25)
knn_k1.fit(X_train_scaled, y_train)

# Predicting Danah's classification with the corrected setup
y_pred = knn_k1.predict_proba(danah_features_scaled)
y_pred


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[1., 0.]])

In [ ]:
from sklearn.metrics import precision_score, recall_score

for k in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
  knn_k = KNeighborsClassifier(n_neighbors=k)
  knn_k.fit(X_train_scaled, y_train)
  y_pred = knn_k.predict(X_test_scaled)
  pscore = precision_score(y_test, y_pred).round(4)
  rscore = recall_score(y_test, y_pred).round(4)
  f1score = recall_score(y_test, y_pred).round(4)
  print(k,pscore,rscore,f1score)

1 0.8851 0.7333 0.7333
2 0.9677 0.5714 0.5714
3 0.973 0.6857 0.6857
4 0.9683 0.581 0.581
5 0.9714 0.6476 0.6476
6 0.9821 0.5238 0.5238
7 0.9538 0.5905 0.5905
8 1.0 0.5333 0.5333
9 0.9851 0.6286 0.6286
10 1.0 0.5238 0.5238
11 0.9841 0.5905 0.5905
12 0.9828 0.5429 0.5429
13 0.9677 0.5714 0.5714
14 0.9831 0.5524 0.5524
15 0.9375 0.5714 0.5714


In [ ]:


from sklearn.metrics import precision_score, recall_score

for k in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
  knn_k = KNeighborsClassifier(n_neighbors=k)
  knn_k.fit(X_train_scaled, y_train)
  #y_pred = knn_k.predict(X_test_scaled)
  y_pred = (knn_k.predict_proba(X_test_scaled)[:,1] >= 0.1).astype(bool)
  pscore = precision_score(y_test, y_pred).round(4)
  rscore = recall_score(y_test, y_pred).round(4)
  f1score = recall_score(y_test, y_pred).round(4)
  print(k,pscore,rscore,f1score)

1 0.8851 0.7333 0.7333
2 0.7963 0.819 0.819
3 0.7323 0.8857 0.8857
4 0.708 0.9238 0.9238
5 0.6803 0.9524 0.9524
6 0.6579 0.9524 0.9524
7 0.593 0.9714 0.9714
8 0.5628 0.981 0.981
9 0.5337 0.981 0.981
10 0.5099 0.981 0.981
11 0.695 0.9333 0.9333
12 0.6759 0.9333 0.9333
13 0.6667 0.9333 0.9333
14 0.6452 0.9524 0.9524
15 0.6211 0.9524 0.9524
